## Out methods
1) Try raw jammin scores
2) Try normalized jammin scores
3) Normalized jammin + vader scores 
4) Balance dataset

In [1]:
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import requests
import json

from random import shuffle

import numpy as np

import time

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn import preprocessing



from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

import scipy


ImportError: No module named 'nltk'

In [2]:
def jammin_emotion(text):
    # Pre process text
    text = text.replace('\\"','"')
    text = text.replace('"','\\"')
    text = text.replace("\n"," ")
    text = text.replace('â€™',"'")
    text = text.replace('â€œ','\\"')
    text = text.replace('â€','\\"')
    text = text.replace('â€”','-')
    text = text.replace('â€¦','...')
    text = text.replace('ðŸ¤”',' ')
    text = text.replace('Ÿ™„',' ')
    text = text.replace('â€˜',"'")
    text = text.replace('ðŸ‘',' ')
    text = text.replace('ðŸ˜‰',' ')
    text = text.replace('ðŸ¤¨',' ')
    text = text.replace('ðŸ˜„',' ')
    text = text.replace('Ÿ˜‚',' ')
    text = text.replace('Ÿ˜',' ')
    text = text.replace('€‹',' ')
    text = text.replace('€‹',' ')
    text = text.replace('„',' ')
    text = text.replace('Ÿ',' ')
    text = text.replace('€',' ')
    text = text.replace('™‚',' ')
    text = text.replace('’',' ')
    text = text.replace('™',' ')
    text = text.replace('‡',' ')
    text = text.replace('‘Š',' ')
    text = text.replace('–',' ')
    text = text.replace('“',' ')
    text = text.replace('”',' ')
    text = text.replace('š',' ')
    text = text.replace('˜',' ')
    text = text.replace('œ…',' ')
    text = text.replace('œ',' ')
    text = text.replace('‹',' ')
    text = text.replace('Œ',' ')
    text = text.replace('›',' ')
    text = text.replace('‘',' ')
    text = text.replace('…',' ')
    text = text.replace('Ž',' ')
    
    print(text)
 
    payload = '{"text":"%s","lang":"en"}'%(text)
    #print payload
    headers = {'content-Type': 'application/json'}

    url = 'http://1.34.96.63:8080/webresources/jammin/emotion'
    #url = 'http://127.0.0.1:8080/JamminTextEmotionAPI/webresources/jammin/emotion'

    r = requests.post(url,  data=payload, headers=headers)
    #print(r)
    return r.json()

def contains_a_d_or_b(response):
    if "text" in response:
        if "yes" in response["bullying"]:
            #print "Contains bullying"
            return True
        for group in response["groups"]:
            if "anger" in group["name"]:
                #print "Contains anger"
                return True

            if "disgust" in group["name"]:
                #print "Contains disgust"
                return True
    return False


def get_negativity_ind_posts(post,snr,  an, ac):
    vector = []

    # Count the number of negative sentences
    count_neg_sents = 0
    neg_ratio = 0.0
    count_bullying = 0
    bullying_ratio = 0.0

    negs = []
    compounds = []

    # To get the average sentiment scores
    tot_sentences = len(post["sentences"])
    
    # To determine if this comment contains negativity
    neg_comment = False
    
    for sent in post["sentences"]:

        # Get the Vader scores
        vs = sent["vader"]
        neg = float(vs["neg"])
        compound = float(vs["compound"])

        negs.append(neg)
        compounds.append(compound)
        # Add the negativity and compound
        #sum_neg = sum_neg + neg
        #sum_compound = sum_compound + compound

        # Get the Jammin scores
        je = sent["jammin"]

        # If both jammin and vader detect negativity
        if contains_a_d_or_b(je) and (neg >= 0.05 or compound <= -0.05):
            # Mark this comment as being negative
            neg_comment = True

            count_neg_sents = count_neg_sents + 1
            if "yes" in je["bullying"]:
                count_bullying = count_bullying + 1

    # Flag the post a negative
    if neg_comment:
        vector.append(1.0)
    else:
        vector.append(0.0)
        

    #avg_neg = float(sum_neg) / float(tot_sentences)
    #avg_compound = float(sum_compound) / float(tot_sentences)

    avg_neg = np.mean(negs)
    avg_compound = np.mean(compounds)
    
    vector.append(avg_neg)
    vector.append(avg_compound)

    #if tot_sentences > 0:
    # Percentage of neg comments for this post
    #neg_ratio = float(count_neg) / float(tot_sentences)
    # Percentage of neg sentences for this post
    sent_neg_ratio = float(count_neg_sents) / float(tot_sentences)
    # Percentage of bullying detectec on this post
    bullying_ratio = float(count_bullying) / float(tot_sentences)
    
    vector.append(sent_neg_ratio)
    vector.append(bullying_ratio)

    if sent_neg_ratio >= snr and (avg_neg >= an or avg_compound <= -ac):
        # Determine the level
        level = 1

    return vector


In [3]:
sentence = "Excellent reminder of where things could go! California and Sanctuary cities do not speak for the rest of Americans anymore than the KKK and racial laws during the Civil Rights movements.and the illegal laws that accompanied them! Illegal immigrants do not belong here, they are illegal. If immigrants want protection, and then do what you need to become legal and then I will welcome them!"
sentence = ' " I love how they are so "concerned" about gun control.'

In [4]:
print (jammin_emotion(sentence))

 \" I love how they are so \"concerned\" about gun control.


NameError: name 'requests' is not defined

In [5]:
analyzer = SentimentIntensityAnalyzer()

NameError: name 'SentimentIntensityAnalyzer' is not defined

In [6]:
vs = analyzer.polarity_scores(sentence)
print (vs)

NameError: name 'analyzer' is not defined

In [7]:
json_samples = []

#with open('../twitter-hatespeech/data/data_manual.json') as json_file:  
with open('../Classifier/data_manual_axel_plus_turk.json') as json_file:
    data = json.load(json_file)
    
    
    
    for i, p in enumerate(data):
        text = p['text']
        
        
        if i < 53:
            continue
            
        if i == 1197:
            continue
        #if i > 10:
        #    break
        
        print(i)
        # List to store the sentences and their emotions
        sentences = []
        
        # Split into sentences
        tokenized_sentences = nltk.sent_tokenize(text)
        
        found = False
        for sentence in tokenized_sentences:
            sentence = sentence.strip()
            # Get Vader sentiment
            #sentence = sentence.encode('utf-8')
            #sentence = bytes(sentence, 'utf-8').decode('utf-8', 'ignore')
            print (sentence)
            #print()
            
            res = jammin_emotion(sentence)
            
            vs = analyzer.polarity_scores(sentence)
            
            
            # Add the sentence data to the list
            sent_data = {"text":sentence,"vader":vs,"jammin":res}
            #sent_data = {"text":sentence,"vader":vs}
            sentences.append(sent_data)
            #count = count + 1
        
        #print (sentences)
        p["sentences"] = sentences
        json_samples.append(p)
        
        #print('')

with open("data_manual_axel_plus_turk_sentiment.json","w") as outfile:  
    json.dump(json_samples, outfile)
    outfile.close()

FileNotFoundError: [Errno 2] No such file or directory: '../Classifier/data_manual_axel_plus_turk.json'

## Determine the emotions to include as part of the binary vector

In [8]:
f = open("features.txt", "r")
features = {}
for line in f:
    line = line.strip()
    if not "#" in line and len(line) > 0:
        features[line] = line



emotion_count = features
for emotion in emotion_count:
    emotion_count[emotion] = 0
print(emotion_count)
print(len(emotion_count))

{'gross': 0, 'regret': 0, 'numb': 0, 'angry': 0, 'bullying': 0, 'nauseous': 0, 'horrible': 0, 'aggravated': 0, 'awful': 0, 'bored': 0, 'furious': 0, 'bummed': 0, 'aggressiveness': 0, 'disgust': 0, 'annoyed': 0, 'hungover': 0, 'blah': 0, 'annoyance': 0, 'dumb': 0, 'anger': 0, 'mad': 0, 'fedup': 0, 'shame': 0, 'frustrated': 0, 'disgusted': 0, 'terrible': 0, 'rage': 0, 'remorse': 0, 'lousy': 0, 'meh': 0, 'defeated': 0, 'cheated': 0, 'stupid': 0, 'lame': 0, 'pissedoff': 0, 'anger2': 0, 'ugly': 0, 'pissed': 0, 'boredom': 0, 'rough': 0, 'insulted': 0, 'grumpy': 0, 'offended': 0, 'contempt': 0, 'enraged': 0, 'bitter': 0, 'yucky': 0, 'ill': 0, 'irritated': 0}
49


In [9]:
print(emotion_count)

{'gross': 0, 'regret': 0, 'numb': 0, 'angry': 0, 'bullying': 0, 'nauseous': 0, 'horrible': 0, 'aggravated': 0, 'awful': 0, 'bored': 0, 'furious': 0, 'bummed': 0, 'aggressiveness': 0, 'disgust': 0, 'annoyed': 0, 'hungover': 0, 'blah': 0, 'annoyance': 0, 'dumb': 0, 'anger': 0, 'mad': 0, 'fedup': 0, 'shame': 0, 'frustrated': 0, 'disgusted': 0, 'terrible': 0, 'rage': 0, 'remorse': 0, 'lousy': 0, 'meh': 0, 'defeated': 0, 'cheated': 0, 'stupid': 0, 'lame': 0, 'pissedoff': 0, 'anger2': 0, 'ugly': 0, 'pissed': 0, 'boredom': 0, 'rough': 0, 'insulted': 0, 'grumpy': 0, 'offended': 0, 'contempt': 0, 'enraged': 0, 'bitter': 0, 'yucky': 0, 'ill': 0, 'irritated': 0}


In [10]:
keys = emotion_count.keys()
print(keys)

dict_keys(['gross', 'regret', 'numb', 'angry', 'bullying', 'nauseous', 'horrible', 'aggravated', 'awful', 'bored', 'furious', 'bummed', 'aggressiveness', 'disgust', 'annoyed', 'hungover', 'blah', 'annoyance', 'dumb', 'anger', 'mad', 'fedup', 'shame', 'frustrated', 'disgusted', 'terrible', 'rage', 'remorse', 'lousy', 'meh', 'defeated', 'cheated', 'stupid', 'lame', 'pissedoff', 'anger2', 'ugly', 'pissed', 'boredom', 'rough', 'insulted', 'grumpy', 'offended', 'contempt', 'enraged', 'bitter', 'yucky', 'ill', 'irritated'])


In [11]:
def all_data_to_vector(output, sentiment = True, thresholds = True, normalized=True):
    vectors = []
    with open("data_manual_axel_plus_turk_sentiment.json") as json_file:  
        data = json.load(json_file)
        
        for post in data:
            emotions_map = emotion_count.copy()
            if sentiment:
               
                emotions_map["neg"] = 0
                emotions_map["neu"] = 0
                emotions_map["pos"] = 0
                emotions_map["compound"] = 0
            num_sentences = len(post["sentences"])
            for sentence in post["sentences"]:
                if sentiment:
                    vader = sentence["vader"]
                    #print (vader)
                    emotions_map["neg"] += vader["neg"]
                    emotions_map["neu"] += vader["neu"]
                    emotions_map["pos"] += vader["pos"]
                    emotions_map["compound"] += vader["compound"] 
                    
                groups = sentence["jammin"]["groups"]
                for group in groups:
                    if "anger" in group["name"] or "disgust" in group["name"]:
                        for emotion in group["emotions"]:
                            if emotion in emotions_map:
                                emotions_map[emotion] += 1.0

            vector = []
            if normalized:
                for count in emotions_map.values():
                    vector.append(count/float(num_sentences))
            else:
                for count in emotions_map.values():
                    vector.append(count)
                    
            if thresholds:
                vector = vector + get_negativity_ind_posts(post,0,  0, 0)

            vectors.append(vector)
    with open(output,"w") as outfile:  
        json.dump(vectors, outfile)
        outfile.close()
    print(emotions_map)
    return vectors

def some_data_to_vector(indices, output, sentiment = True, normalized=True):
    vectors = []
    with open("data_manual_sentiment.json") as json_file:  
        data = json.load(json_file)
        
        for index in indices:
            post = data[index]
            emotions_map = emotion_count.copy()
            if sentiment:
               
                emotions_map["neg"] = 0
                emotions_map["neu"] = 0
                emotions_map["pos"] = 0
                emotions_map["compound"] = 0
            num_sentences = len(post["sentences"])
            for sentence in post["sentences"]:
                if sentiment:
                    vader = sentence["vader"]
                    #print (vader)
                    emotions_map["neg"] += vader["neg"]
                    emotions_map["neu"] += vader["neu"]
                    emotions_map["pos"] += vader["pos"]
                    emotions_map["compound"] += vader["compound"] 
                    
                groups = sentence["jammin"]["groups"]
                for group in groups:
                    if "anger" in group["name"] or "disgust" in group["name"]:
                        for emotion in group["emotions"]:
                            if emotion in emotions_map:
                                emotions_map[emotion] += 1.0

            vector = []
            for count in emotions_map.values():
                vector.append(count/float(num_sentences))

            vectors.append(vector)
    with open(output,"w") as outfile:  
        json.dump(vectors, outfile)
        outfile.close()
    print(emotions_map)
    return vectors

'''def all_data_to_vector2(output, sentiment=True, emotions=["anger","disgust"], normalized=True):
    vectors = []
    with open("data_manual_sentiment.json") as json_file:  
        data = json.load(json_file)
        
        for post in data:
            emotions_map = emotion_count.copy()
            if sentiment:
                print ("Including sentiment")
                emotions_map["neg"] = 0
                emotions_map["neu"] = 0
                emotions_map["pos"] = 0
                emotions_map["compound"] = 0
            
            num_sentences = len(post["sentences"])
            for sentence in post["sentences"]:
                # Sentiment section
                if sentiment:
                    vader = sentence["vader"]
                    emotions_map["neg"] += vader["neg"]
                    emotions_map["neu"] += vader["neu"]
                    emotions_map["pos"] += vader["pos"]
                    emotions_map["compound"] += vader["compound"]
                    
                
                # Emotion section
                groups = sentence["jammin"]["groups"]
                # For each emotion group
                for group in groups:
                    # For each emotion to include
                    #for emo in emotions:
                        # If the group name matches an emotion to include
                        if "anger" in group["name"]:
                            # If a subemotion within the group is in the emotion map
                            for emotion in group["emotions"]:
                                if emotion in emotions_map:
                                    emotions_map[emotion] += 1.0

            for count in emotions_map.values():
                vector.append(count/num_sentences)

            vectors.append(vector)
            print (vector)
    with open(output,"w") as outfile:  
        json.dump(vectors, outfile)
        outfile.close()
    return vectors
'''

def single_text_to_vector(text, sentiment = True, normalized=True, thresholds=True):
    emotions_map = emotion_count.copy()
    if sentiment:
        emotions_map["neg"] = 0
        emotions_map["neu"] = 0
        emotions_map["pos"] = 0
        emotions_map["compound"] = 0
            
    # Split into sentences
    tokenized_sentences = nltk.sent_tokenize(text)
    num_sentences = len(tokenized_sentences)
    for sentence in tokenized_sentences:
        if sentiment:
            vader = analyzer.polarity_scores(sentence)
            print (vader)
            emotions_map["neg"] += vader["neg"]
            emotions_map["neu"] += vader["neu"]
            emotions_map["pos"] += vader["pos"]
            emotions_map["compound"] += vader["compound"] 
                
        res = jammin_emotion(sentence)
        print (res)
        print()
        groups = res["groups"]
        for group in groups:
            if "anger" in group["name"] or "disgust" in group["name"]:
                for emotion in group["emotions"]:
                    if emotion in emotions_map:
                        emotions_map[emotion] += 1.0

    vector = []
    if normalized:
        for count in emotions_map.values():
            vector.append(count/float(num_sentences))
    else:
        for count in emotions_map.values():
            vector.append(count)
                    
    #if thresholds:
        #vector = vector + get_negativity_ind_posts(text,0,  0, 0)
            
    return [vector]
    
    

        
                            
        
                    
            
            

In [12]:
output = "features_vader_jammin(disgust_anger)_norm.json"
#input_file = "data_manual_sentiment.json"
vectors = all_data_to_vector( output, normalized=False)

NameError: name 'json' is not defined

In [13]:

print(vectors[5])


NameError: name 'vectors' is not defined

In [14]:
vectorx = single_text_to_vector("Obama is a Muslim and was born in Africa. His whole mission is to start the downfall of America by allowing the invasion of illegal immigrants and Muslim terrorist!",sentiment = True, normalized=False)

NameError: name 'nltk' is not defined

In [15]:
 print(vectorx)

NameError: name 'vectorx' is not defined

In [16]:
0: "horrible" 1
1: "ugly" 1
2: "disgusted" 2 
    
0: "meh" 1
1: "blah" 1 
2: "bitter" 1
    

1: "defeated" 1
2: "dumb" 1
3: "terrible" 1
    
0: "grumpy" 1
1: "angry" 1
2: "fedup" 1

SyntaxError: invalid syntax (<ipython-input-16-56a47ee1d6d9>, line 1)

## Testing SVM

https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [17]:
indices_1 = []
indices_0 = []

with open("data_manual_axel_plus_turk_sentiment.json") as json_file:  
    data = json.load(json_file)
    X = []
    y = []
    for index, post in enumerate(data):
        text = post['text']
        X.append(text)
        
        if "no_hs" in post["label"]:
            #if len(indices_0) < 330:
            y.append(0)
            indices_0.append(index)
        else:
            y.append(1)
            indices_1.append(index)
print(y.count(0))
print(y.count(1))
print(len(indices_0))
print(len(indices_1))
#indices = indices_0 + indices_1
#output = "./some_features_vader_jammin(disgust_anger)_norm.json"
#vectors = some_data_to_vector(indices, output)

vectors = all_data_to_vector( output, sentiment = True, thresholds = False, normalized=False)

NameError: name 'json' is not defined

In [18]:
print(len(vectors))

NameError: name 'vectors' is not defined

In [19]:
#X = [[0], [1], [2], [3]]
#Y = [0, 1, 2, 3]
scoring = ['accuracy','precision', 'recall','f1',
           'precision_micro', 'precision_macro', 'precision_weighted',
           'recall_micro', 'recall_macro', 'recall_weighted',
           'f1_micro','f1_macro','f1_weighted']

c = svm.SVC(kernel='linear', C=1, random_state=0)
#c = svm.SVC(C=1, kernel='linear')

Cs = np.logspace(-6, 3, 10)
grid={"C":Cs, "penalty":["l1","l2"]}# l1 lasso l2 ridge
#grid={"C":Cs}
c=LogisticRegression()
clf = GridSearchCV(estimator=c, param_grid=grid,n_jobs=-1)
clf.fit(vectors, y) 

normalized_X = preprocessing.normalize(vectors)
scores = cross_validate(clf, normalized_X, y, cv=10, scoring=scoring)
print(scores)
print(clf.best_score_)                              
print(clf.best_estimator_.C)




NameError: name 'svm' is not defined

In [20]:
printScores(scores)

NameError: name 'printScores' is not defined

## Testing

In [21]:
vector = single_text_to_vector("You are such a piece of shit. I will beat the crap out of you. You have such a horrible face. Eat shit bitch!")
print (vector)

NameError: name 'nltk' is not defined

In [22]:
clf.predict(vector)

NameError: name 'clf' is not defined

## Averaging results

In [23]:
from statistics import mean
test_precision_micro = mean([0.48148148, 0.48148148, 0.51851852, 0.54074074, 0.56296296,
       0.49253731, 0.56716418, 0.52238806, 0.59701493, 0.58208955])

test_precision_macro = mean([0.48070919, 0.48094816, 0.51837588, 0.54644525, 0.56333333,
       0.49247698, 0.56740443, 0.52457814, 0.61754386, 0.59664306])

test_recall_micro = mean([0.48148148, 0.48148148, 0.51851852, 0.54074074, 0.56296296,
       0.49253731, 0.56716418, 0.52238806, 0.59701493, 0.58208955])

test_recall_macro = mean([0.48101405, 0.48112379, 0.51832748, 0.53928885, 0.56255487,
       0.49253731, 0.56716418, 0.52238806, 0.59701493, 0.58208955])
    
test_f1_micro = mean([0.48148148, 0.48148148, 0.51851852, 0.54074074, 0.56296296,
       0.49253731, 0.56716418, 0.52238806, 0.59701493, 0.58208955])
    
test_f1_macro = mean([0.47916667, 0.48008363, 0.51809545, 0.52160494, 0.56142283,
       0.49151786, 0.56677815, 0.51150604, 0.57861635, 0.56574074])

print("Macro")
print("average precision is %f"%test_precision_macro)
print("average recall is %f"%test_recall_macro)
print("average f1 is %f"%test_f1_macro)

print("Micro")
print("average precision is %f"%test_precision_micro)
print("average recall is %f"%test_recall_micro)
print("average f1 is %f"%test_f1_micro)

Macro
average precision is 0.538846
average recall is 0.534350
average f1 is 0.527453
Micro
average precision is 0.534638
average recall is 0.534638
average f1 is 0.534638


## Prepare data for turkers

In [24]:
with open("data_manual_sentiment.json") as json_file:  
    data = json.load(json_file)
    count = 0
    
    out = open("turk.csv","w")
    out.write("source,sample\n")
    for index, post in enumerate(data):
        if not "no_hs" in post["label"]:
            
            if len(post["text"]) <= 1000:
                count += 1
                text = post["text"] 
                text = text.replace(","," ")
                # Pre process text
                text = text.replace('"','\\"')
                text = text.replace("\n"," ")
                text = text.replace('â€™',"'")
                text = text.replace('â€œ','\\"')
                text = text.replace('â€','\\"')
                text = text.replace('â€”','-')
                text = text.replace('â€¦','...')
                text = text.replace('ðŸ¤”',' ')
                text = text.replace('Ÿ™„',' ')
                text = text.replace('â€˜',"'")
                text = text.replace('ðŸ‘',' ')
                text = text.replace('ðŸ˜‰',' ')
                text = text.replace('ðŸ¤¨',' ')
                text = text.replace('ðŸ˜„',' ')
                text = text.replace('Ÿ˜‚',' ')
                text = text.replace('Ÿ˜',' ')
                text = text.replace('€‹',' ')
                text = text.replace('€‹',' ')
                text = text.replace('„',' ')
                text = text.replace('Ÿ',' ')
                text = text.replace('€',' ')
                text = text.replace('™‚',' ')
                text = text.replace('’',' ')
                text = text.replace('™',' ')
                text = text.replace('‡',' ')
                text = text.replace('‘Š',' ')
                text = text.replace('–',' ')
                text = text.replace('“',' ')
                text = text.replace('”',' ')
                text = text.replace('š',' ')
                text = text.replace('˜',' ')
                text = text.replace('œ…',' ')
                text = text.replace('œ',' ')
                text = text.replace('‹',' ')
                text = text.replace('Œ',' ')
                text = text.replace('›',' ')
                text = text.replace('‘',' ')
                text = text.replace('…',' ')
                text = text.replace('Ž',' ')
                #print(post["text"])
            #else: 
                #print("*"*10+str(index)+"*"*10)
                #print(len(post["text"]))
                out.write(str(index)+ ", \""+ text+"\"\n")
            
    out.close()
    print(count)

NameError: name 'json' is not defined

### Second Additions - After checking paper 2 and noticying it killed us

#### Features extraction

In [25]:
indices_1 = []
indices_0 = []

with open("data_manual_axel_plus_turk_sentiment.json") as json_file:  
    data = json.load(json_file)
    X = []
    y = []
    for index, post in enumerate(data):
        text = post['text']
        X.append(text)
        
        if "no_hs" in post["label"]:
            #if len(indices_0) < 330:
            y.append(0)
            indices_0.append(index)
        else:
            y.append(1)
            indices_1.append(index)
print(y.count(0))
print(y.count(1))
print(len(indices_0))
print(len(indices_1))
#indices = indices_0 + indices_1
#output = "./some_features_vader_jammin(disgust_anger)_norm.json"
#vectors = some_data_to_vector(indices, output)

vectors = all_data_to_vector( output, sentiment = True, thresholds = False, normalized=False)
print(len(vectors))

NameError: name 'json' is not defined

In [26]:
count_vect = CountVectorizer(ngram_range = (1, 1))
X_train_counts = count_vect.fit_transform(X)
print(X_train_counts.shape)

#tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
#X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf = X_train_counts
print(X_train_tf.shape)

X=X_train_tf.tocoo()

X = X.todense().tolist()

for i,row in enumerate(X):
    vectors[i] = vectors[i] + row

print(len(vectors[9]))

vectors = scipy.sparse.csr_matrix(vectors)
print(type(vectors))

NameError: name 'CountVectorizer' is not defined

In [27]:
print(type(X_train_tf))

NameError: name 'X_train_tf' is not defined

In [28]:
X=X_train_tf.tocoo()

NameError: name 'X_train_tf' is not defined

In [29]:
X.data

NameError: name 'X' is not defined

In [30]:
X.row

NameError: name 'X' is not defined

In [31]:
X.col

NameError: name 'X' is not defined

In [32]:
X = X.todense().tolist()

NameError: name 'X' is not defined

In [33]:
print(X)

NameError: name 'X' is not defined

In [34]:
print(X[0][1318])

NameError: name 'X' is not defined

In [35]:
for i,row in enumerate(X):
    vectors[i] = vectors[i] + row

NameError: name 'X' is not defined

In [36]:
print(len(vectors[9]))

NameError: name 'vectors' is not defined

In [37]:
vectors = scipy.sparse.csr_matrix(vectors)
print(type(vectors))

NameError: name 'scipy' is not defined

In [38]:
scoring = ['accuracy','precision', 'recall','f1',
           'precision_micro', 'precision_macro', 'precision_weighted',
           'recall_micro', 'recall_macro', 'recall_weighted',
           'f1_micro','f1_macro','f1_weighted']

#c = svm.SVC(kernel='linear', C=1, random_state=0)
#c = svm.SVC(C=1, kernel='linear')
c = SGDClassifier(loss='hinge', penalty='l2', # hinge gives an SVM
                                           alpha=1e-3, random_state=42,
                                           max_iter=5, tol=None)
Cs = np.logspace(-6, 3, 10)
#grid={"C":Cs, "penalty":["l1","l2"]}# l1 lasso l2 ridge
grid={"alpha":(1e-1, 1e-2, 1e-3, 1e-4), "penalty":["l1","l2"]}# l1 lasso l2 ridge
#grid={"C":Cs}
#c=LogisticRegression()

vectors = preprocessing.normalize(vectors)
clf = GridSearchCV(estimator=c, param_grid=grid,n_jobs=-1)
clf.fit(vectors, y) 


scores = cross_validate(clf, vectors, y, cv=10, scoring=scoring)
print(scores)
print(clf.best_score_)                              
#print(clf.best_estimator_.C)

NameError: name 'SGDClassifier' is not defined

In [39]:
for param_name in sorted(grid.keys()):
    print("%s: %r" % (param_name, clf.best_params_[param_name]))

NameError: name 'grid' is not defined

In [40]:
def printScores(scores):
    from statistics import mean
    test_precision_micro = mean(scores['test_precision_micro'])

    test_precision_macro = mean(scores['test_precision_macro'])

    test_recall_micro = mean(scores['test_recall_micro'])

    test_recall_macro = mean(scores['test_recall_macro'])

    test_f1_micro = mean(scores['test_f1_micro'])

    test_f1_macro = mean(scores['test_f1_macro'])

    print("Macro")
    print("average precision is %f"%test_precision_macro)
    print("average recall is %f"%test_recall_macro)
    print("average f1 is %f"%test_f1_macro)

    print("Micro")
    print("average precision is %f"%test_precision_micro)
    print("average recall is %f"%test_recall_micro)
    print("average f1 is %f"%test_f1_micro)
printScores(scores)

NameError: name 'scores' is not defined

In [41]:
def jammin_emotion(text):
   
    payload = '{"text":"%s","lang":"en"}'%(text)
    #print payload
    headers = {'content-Type': 'application/json'}

    url = 'http://1.34.96.63:8080/webresources/jammin/emotion'
    #url = 'http://127.0.0.1:8080/JamminTextEmotionAPI/webresources/jammin/emotion'

    r = requests.post(url,  data=payload, headers=headers)
    #print(r)
    return r.json()

jammin_emotion("I am sorry that I was too busy to meet, but I wish you all the best today!")

NameError: name 'requests' is not defined